## Τεχνητή Νοημοσύνη: Εργαστηριακή Άσκηση 2
## Ονοματεπώνυμο: Θεοδοσίου Ιωάννης
## 03115056

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
import os
os.listdir('/gdrive/My Drive')
path='/gdrive/My Drive/'

Mounted at /gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#install swi-prolog
!sudo apt-get install software-properties-common
!sudo apt-add-repository ppa:swi-prolog/stable
!sudo apt-get update
!sudo apt-get install swi-prolog

Reading package lists... Done
Building dependency tree       
Reading state information... Done
software-properties-common is already the newest version (0.96.24.32.11).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
 Comprehensive Prolog implementation with extensive libraries and development tools.   Primarily targetted at teaching, RDF processing and web-related tasks, such as creating web services or analysing web content.
 More info: https://launchpad.net/~swi-prolog/+archive/ubuntu/stable
Press [ENTER] to continue or Ctrl-c to cancel adding it.

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bio

In [ ]:
#install pyswip
!pip install pyswip

In [ ]:
# Η βιβλιοθήκη pandas είναι χρήσιμη για την εργασία με τέτοια δεδομένα
import pandas as pd 
from pyswip import Prolog
import re
# Διάβασμα του αρχείου 'movie_metadata.csv' 
data = pd.read_csv("/content/drive/My Drive/AI_ex2/movie_metadata.csv") 
#Στο csv υπαρχούν κελία με nan τιμές
#Στις θέσεις αυτές βάζουμε 'UNK' πράγμα που  κάνουμε με την παρακάτω συνάρτηση
data.fillna("UNK", inplace=True)
# Preview the first 5 lines of the loaded data 
data.head()
def clean_text(text):
    text = text.replace(u'\xa0', u'')
    text = text.replace(u"'", u'')
    text = text.replace(u'"', u'')
    return text

## ΜΕΡΟΣ ΠΡΩΤΟ 
Σκοπός της εργαστηριακής αυτής άσκησης είναι η δημιουργία λογικών κανόνων για movie recomendation. Αρχικά αφου διαβάσουμε το αρχείο με τα δεδομένα μας δημιουργούμε τον κοσμο στον οποίο θα εργαστούμε και τα βασικά κατηγορήματα τα οποία είναι της μορφής:

1. director(movie,D)
2. language(movie,L)
3. color(movie,C)
4. actor(movie,A)

Τους ηθοποίους τους αντιμετωπίσαμε ως το ίδιο κατηγόρημα και δεν διακρίναμε κάποια σειρά όπως πρώτος δεύτερος και τρίτος ρόλος.

Επίσης στα κατηγορήματα μας πραγαμτοποιούμε μία sort καθώς η prolog θέλει όλα τα κατηγορήματα της ταξινομημένα.

In [ ]:
#create World
#Ορίζουμε τον κόσμο μας
prolog = Prolog()
#και σβήνουμε ότι υπήρχε αποθηκευμένο από πρίν
prolog.query('make')
movie_names=[]
#Για κάθε row του πίνακα φτιάχνουμε τα κατηγοτήματα που θέλουμε να αποθηκέυσουμε
#αρχικά σε μια λίστα με το όνομα literals
literals = []
movie_score = {}
for row in data.itertuples(index=True, name='Pandas'):
    movie_title = clean_text(getattr(row, 'movie_title'))
    director = clean_text(getattr(row, 'director_name'))

    color=clean_text(getattr(row,'color'))
    plot=clean_text(getattr(row,'plot_keywords'))
    language=clean_text(getattr(row,'language'))
    actor1=clean_text(getattr(row,'actor_1_name'))
    actor2=clean_text(getattr(row,'actor_2_name'))
    actor3=clean_text(getattr(row,'actor_3_name'))
    
    movie_score[movie_title] = 0
    movie_title = movie_title.rstrip()

    movie_names.append(movie_title)
    
    if movie_title != "UNK":
        if director != "UNK":
            literals.append("director('"+ movie_title +"','" + director + "')")
        for genre in getattr(row, 'genres').split("|"):
            literals.append("genre('"+ movie_title +"','"+ genre +"')")
        for pl in plot.split("|"):
          if pl!="UNK":
            literals.append("plot('"+ movie_title +"','"+ pl +"')")
        if actor1 != "UNK":
            literals.append("actor('"+ movie_title +"','" + actor1 + "')")
        if actor2 != "UNK":
            literals.append("actor('"+ movie_title +"','" + actor2 + "')")
        if actor3 != "UNK":
            literals.append("actor('"+ movie_title +"','" + actor3 + "')")
        if language != "UNK":
            literals.append("language('"+ movie_title +"','" + language + "')")
        if color != "UNK":
            literals.append("color('"+ movie_title +"','" + color + "')")
        

#Η Prolog θέλει τα κατηγορήματά της με την σειρά 
literals.sort()
for literal in literals:
    prolog.assertz(literal)

#Επίσης μπορούμε να κάνουμε consult ένα έτοιμό αρχείο στον κόσμο όπως γίνεται παρακάτω
# prolog.consult("filename.pl")

In [ ]:
print(literals)

## ΜΕΡΟΣ 1Β
Αφού δημιουργήσαμε τον κόσμο μας τώρα δημιουργούμε βασικούς κανόνες οι οποίοι βρίσκουν όμοιες ταινίες με βάση κάποιο συγκεκριμένο χαρακτηριστικό. Οί κανόνες που δημιουργήθηκαν είναι οι εξής:

1. κοινή γλώσσα find_same_language(X,Y)
2. κοινός σκηνοθέτης find_same_director(X,Y)
3. κοινό χρώμα find_same_color(X,Y)
4. πολύ κοινή πλοκή με 3 κοινά plot words fully_common_plot(X,3,Y)
5. μετρίως κοινή πλοκή με 2 κοινά plot words medium_common_plot(X,2,Y)
6. λίγο κοινή πλοκή με 1 κοινό plot word less_common_plot(X,1,Y)
7. πολύ κοινό είδος με 4 κοινά genre words fully_common_genre(X,4,Y)
8. μετρίως κοινό είδος με 3 κοινά genre words medium_common_genre(X,3,Y)
9. λιγότερο κοινό είδος με 2 κοινα genre word less_common_genre(X,2,Y)
10. 3 κοινοί ηθοποιοί actor_common_3(X,3,Y)
11. 2 κοινοί ηθοποιοί actor_common_2(X,2,Y)
12. 1 κοινός ηθοποιός actor_common_1(X,1,Y)

Για το πλήθος των διαφορετικών ηθοποιών, plot words και genre words χρισημοποιήθηκαν οι βοηθητικοί κανόνες:

common_plot_points(X,Y,N),common_genre_points(X,Y,N) και common_actor_points(X,Y,N) οι οποίοι για κάθε ζεύγος ταινιών Χ,Υ επιστρέφουν το πλήθος των κοινών plot words, genre words και ηθοποιών αντίστοιχα.

In [ ]:
#completed rules
prolog.assertz('(find_same_language(X,Y) :- language(X,L),language(Y,L),X\=Y)')
prolog.assertz('(find_same_director(X,Y) :- director(X,L),director(Y,L),X\=Y)')
prolog.assertz('(find_same_color(X,Y) :- color(X,L),color(Y,L),X\=Y)')

prolog.assertz('common_plot_points(X,Y,N) :- findall(P1,plot(X,P1),L1),findall(P2,plot(Y,P2),L2),intersection(L1,L2,L),length(L,N)')
prolog.assertz('fully_common_plot(X,3,L) :- findall(Y,director(Y,_),L1),member(L,L1),common_plot_points(X,L,3)')
prolog.assertz('medium_common_plot(X,2,L) :- findall(Y,director(Y,_),L1),member(L,L1),common_plot_points(X,L,2)')
prolog.assertz('less_common_plot(X,1,L) :- findall(Y,director(Y,_),L1),member(L,L1),common_plot_points(X,L,1)')

prolog.assertz('common_genre_points(X,Y,N) :- findall(G1,genre(X,G1),L1),findall(G2,genre(Y,G2),L2),intersection(L1,L2,L),length(L,N)')
prolog.assertz('fully_common_genre(X,4,L) :- findall(Y,director(Y,_),L1),member(L,L1),common_genre_points(X,L,5)')
prolog.assertz('medium_common_genre(X,3,L) :- findall(Y,director(Y,_),L1),member(L,L1),common_genre_points(X,L,4)')
prolog.assertz('less_common_genre(X,2,L) :- findall(Y,director(Y,_),L1),member(L,L1),common_genre_points(X,L,3)')

prolog.assertz('common_actors_points(X,Y,N) :- findall(A1,actor(X,A1),L1),findall(A2,actor(Y,A2),L2),intersection(L1,L2,L),length(L,N)')
prolog.assertz('actor_common_1(X,1,L) :- findall(Y,director(Y,_),L1),member(L,L1),common_actors_points(X,L,1)')
prolog.assertz('actor_common_2(X,2,L) :- findall(Y,director(Y,_),L1),member(L,L1),common_actors_points(X,L,2)')
prolog.assertz('actor_common_3(X,3,L) :- findall(Y,director(Y,_),L1),member(L,L1),common_actors_points(X,L,3)')





Αφου φτιάξαμε τους παραπάνω βασικούς κανόνες τώρα φτιάχνουμε με βάση αυτά κανόνες recomendations σε κλίμακα απο 1 μέχρι 5 οπου η ομοιότητα τύπου 5 σημαίνει full recomended και ομοιότητα τυπου 1 σημαίνει less recomended.O τρόπος που δημιουργήθηκαν αυτοί οι κανόνες ομοιότητας είναι σχετικά αυθαίρετος και βασίζεται κυρίως σε δοκιμές και έλεγχο με βάση αντίστοιχες προτάσεις του imdb.

In [ ]:
prolog.assertz('(find_similar_movies_5(X, Y) :- ((director(X, D), director(Y, D),genre(X, G), genre(Y, G));fully_common_plot(X,3,Y);fully_common_genre(X,4,Y);actor_common_3(X,3,Y)), X \= Y )')
prolog.assertz('(find_similar_movies_4(X, Y) :- (find_same_director(X,Y);medium_common_genre(X,3,Y);medium_common_plot(X,2,Y)), X \= Y, not(find_similar_movies_5(X, Y)) )')
prolog.assertz('find_similar_movies_1(X,Y) :- find_same_language(X,Y),find_same_color(X,Y)')
prolog.assertz('find_similar_movies_3(X,Y) :- less_common_genre(X,3,Y);less_common_plot(X,1,Y);actor_common_2(X,2,Y)')
prolog.assertz('find_similar_movies_2(X,Y) :- genre(X,G),genre(Y,G),actor_common_1(X,1,Y)')

 ## ΜΕΡΟΣ ΔΕΥΤΕΡΟ
 Η συνάρτηση simple_recomender δέχεται ως όρισμα μια ταινία και προτείνει στο χρήστη αντιστοιχα ταινίες αποκλειστικά με βάση τους 5 βαθμούς ομοιότητας όπως οριστικαν παραπάνω. Για κάθε είδος ομοιότητας κρατήσαμε μόνο τις δέκα πρώτες ταινίες που προτείνονται.

In [ ]:
def simple_recommender(movie):

    q = prolog.query("find_similar_movies_5('" + movie +"',M)")
    ans5 = []
    for soln in q:
        ans5.append(soln["M"])
    ans = set(ans5)
    
    q = prolog.query("find_similar_movies_4('" + movie +"',M)")
    ans4 = []
    for soln in q:
        if soln["M"] not in ans:
            ans4.append(soln["M"])
            ans.add(soln["M"])
    
    q = prolog.query("find_similar_movies_3('" + movie +"',M)")
    ans3 = []
    for soln in q:
        if soln["M"] not in ans:
            ans3.append(soln["M"])
            ans.add(soln["M"])
    
    q = prolog.query("find_similar_movies_2('" + movie +"',M)")
    ans2 = []
    for soln in q:
        if soln["M"] not in ans:
            ans2.append(soln["M"])
            ans.add(soln["M"])
    
    
    q = prolog.query("find_similar_movies_1('" + movie +"',M)")

    ans1 = []
    for soln in q:
        if soln["M"] not in ans4:
            ans1.append(soln["M"])
            ans.add(soln["M"])
    #κάνουμε κάποιο Processing στα παραπάνω αποτελέσματα και επιστρέφουμε τις "όμοιες" ταινίες με φθίνουσα σειρά
    return list(set(ans5))[:10], list(set(ans4))[:10],list(set(ans3))[:10],list(set(ans2))[:10],list(set(ans1))[:10]

Παρακάτω δοκίμαζουμε την συνάρτηση μας simple_rcomender δίνοντας απλά μία ταινία και τυπώνοντας ξεχωριστά δέκα ταινίες (η λιγότερες) απο το κάθε είδος ομοιότητας.

In [ ]:
mov="Star Wars: Episode III - Revenge of the Sith"
r5,r4,r3,r2,r1=simple_recommender(mov)
print("##########################################################################")
print("top movie recomendations for  " + mov)
print("###########################################################################")
for i in r5:
  print(i)
#l=prolog.query("")

##########################################################################
top movie recomendations for  Star Wars: Episode III - Revenge of the Sith
###########################################################################
Star Wars: Episode II - Attack of the Clones
Star Wars: Episode IV - A New Hope
Star Wars: Episode I - The Phantom Menace


In [ ]:
print("##########################################################################")
print("movie similarity 4 recomendations for  " + mov)
print("###########################################################################")
for i in r4:
  print(i)

##########################################################################
movie similarity 4 recomendations for  Star Wars: Episode III - Revenge of the Sith
###########################################################################
G-Force
Underworld: Evolution
Star Wars: Episode V - The Empire Strikes Back
The League of Extraordinary Gentlemen
X-Men 2
Zathura: A Space Adventure
The Helix... Loaded
Beastmaster 2: Through the Portal of Time
Dragonball: Evolution
Pokémon 3: The Movie


In [ ]:
print("##########################################################################")
print("movie similarity 3 recomendations for  " + mov)
print("###########################################################################")
for i in r3:
  print(i)

##########################################################################
movie similarity 3 recomendations for  Star Wars: Episode III - Revenge of the Sith
###########################################################################
Close Encounters of the Third Kind
The Twilight Saga: Breaking Dawn - Part 2
The Dead Zone


In [ ]:
print("##########################################################################")
print("movie similarity 2 recomendations for  " + mov)
print("###########################################################################")
for i in r2:
  print(i)

##########################################################################
movie similarity 2 recomendations for  Star Wars: Episode III - Revenge of the Sith
###########################################################################
Takers
Thor: The Dark World
Charlie and the Chocolate Factory
The Golden Compass
American Heist
1941
Cold Mountain
V for Vendetta
Jumper
Mars Attacks!


In [ ]:
print("##########################################################################")
print("movie similarity 1 recomendations for  " + mov)
print("###########################################################################")
for i in r1:
  print(i)

##########################################################################
movie similarity 1 recomendations for  Star Wars: Episode III - Revenge of the Sith
###########################################################################
The Fighter
Doc Hollidays Revenge
You Dont Mess with the Zohan
Ultraviolet
The Matador
Falcon Rising
Red Tails
8MM
Jekyll and Hyde... Together Again
Legends of Oz: Dorothys Return


In [ ]:
mov="Avatar"
k5,k4,k3,k2,k1=simple_recommender(mov)
print("##########################################################################")
print("top movie recomendations for  " + mov)
print("###########################################################################")
for i in k5:
  print(i)

##########################################################################
top movie recomendations for  Avatar
###########################################################################
True Lies
Terminator 2: Judgment Day
The Abyss
The Terminator
Aliens


In [ ]:
print("##########################################################################")
print("movie similarity 4 recomendations for  " + mov)
print("###########################################################################")
for i in k4:
  print(i)

##########################################################################
movie similarity 4 recomendations for  Avatar
###########################################################################
G-Force
Underworld: Evolution
Star Wars: Episode V - The Empire Strikes Back
The League of Extraordinary Gentlemen
X-Men 2
Zathura: A Space Adventure
The Helix... Loaded
Beastmaster 2: Through the Portal of Time
Dragonball: Evolution
Pokémon 3: The Movie


In [ ]:
print("##########################################################################")
print("movie similarity 3 recomendations for  " + mov)
print("###########################################################################")
for i in k3:
  print(i)

##########################################################################
movie similarity 3 recomendations for  Avatar
###########################################################################
Impostor
Terminator Salvation
Renaissance
Back to the Future
Escape from the Planet of the Apes
Pandorum
Fortress
Æon Flux
Starship Troopers
In Time


In [ ]:
print("##########################################################################")
print("movie similarity 2 recomendations for  " + mov)
print("###########################################################################")
for i in k2:
  print(i)

##########################################################################
movie similarity 2 recomendations for  Avatar
###########################################################################
The Mortal Instruments: City of Bones
The Shaggy Dog
Mystery Men
End of Days
Deep Rising
The Last of the Mohicans
Face/Off
RoboCop 3
Undisputed
Tales from the Crypt: Demon Knight


In [ ]:
print("##########################################################################")
print("movie similarity 1 recomendations for  " + mov)
print("###########################################################################")
for i in k1:
  print(i)

##########################################################################
movie similarity 1 recomendations for  Avatar
###########################################################################
The Fighter
Doc Hollidays Revenge
You Dont Mess with the Zohan
Ultraviolet
The Matador
Falcon Rising
Red Tails
8MM
Jekyll and Hyde... Together Again
Legends of Oz: Dorothys Return


Για τον έλεγχο των αποτελεσμάτων μας χρησιμοποιούμε την έτοιμη συνάρτηση που μας δόθηκε jaccard_similarity που δέχεται δύο λίστες και τυπώνει το πλήθος των στοιχείων της τομής τους προς το πλήθος των στοιχείων της ένωσης τους. Τον έλεγχο τον πραγματοποιούμε με βάση το αρχείο relative movies το οποίο για 100 ταινίες περιέχει κάποια ενδεικτικά recomendations. Αξίζει να σημειώσουμε ωστόσο οτο αναμένουμε χαμηλά ποσοστά ομοιότητας για τους εξής λόγους.
1. Η αυθαίρετη φύση του προβλήματος και ο τρόπος που ορίζει ο καθένας την ομοιότητα
2. για κάθε είδος ομοιότητας κρατάμε μόνο τις 10 πρώτες ταινίες και αγνοούμε άλλες που μπορεί να είναι επίσης πολύ σχετικές με την ταινία μας.
3. Έχουμε πολλά είδη ομοιότητας τα οποία απο κάποιο σημείο και μετά δεν μας δίνουν παρόμοιες ταινίες πχ το movie similarity 1 μας δίνει ταινίες που έχουν ίδιο χρώμα και ίδια γλώσσα με τη δοσμένη το οποίο είναι προφανώς πολύ γενικό.

In [ ]:
def jaccard_similarity(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    intersection = len(set1.intersection(set2))
    union = len(set1) + len(set2) - intersection
    return intersection / union

In [ ]:
data_test = pd.read_csv("/content/drive/My Drive/AI_ex2/relative_movies.csv")
test=data_test.take([0], axis=1).values.tolist()
test_recomendations=data_test.iloc[:,1:].values.tolist()
test_names = [i[0] for i in test ]
test_rec=[i[0].split(',') for i in test_recomendations]



In [ ]:
s=0
for i,n in enumerate(test_names):
  print(i)
  r5,r4,r3,r2,r1=simple_recommender(n)
  
  accuracy=jaccard_similarity(r5, test_rec[i])
  print('intersection length: ',len(list(set(r5).intersection(test_rec[i]))))
  print('jac acc: ',accuracy)
  s+=accuracy
print(s/100)

0
intersection length:  0
jac acc:  0.0
1
intersection length:  0
jac acc:  0.0
2
intersection length:  0
jac acc:  0.0
3
intersection length:  0
jac acc:  0.0
4
intersection length:  1
jac acc:  0.07142857142857142
5
intersection length:  1
jac acc:  0.1
6
intersection length:  1
jac acc:  0.1
7
intersection length:  0
jac acc:  0.0
8
intersection length:  0
jac acc:  0.0
9
intersection length:  4
jac acc:  0.36363636363636365
10
intersection length:  0
jac acc:  0.0
11
intersection length:  2
jac acc:  0.16666666666666666
12
intersection length:  6
jac acc:  0.5454545454545454
13
intersection length:  1
jac acc:  0.05263157894736842
14
intersection length:  1
jac acc:  0.09090909090909091
15
intersection length:  7
jac acc:  0.5384615384615384
16
intersection length:  0
jac acc:  0.0
17
intersection length:  0
jac acc:  0.0
18
intersection length:  0
jac acc:  0.0
19
intersection length:  0
jac acc:  0.0
20
intersection length:  0
jac acc:  0.0
21
intersection length:  0
jac acc:  0.

## ΜΕΡΟΣ ΤΡΙΤΟ 
Τώρα καλούμαστε να δημιουργήσουμε recomendations οχι μόνο με βάση την ομοιότητα των ταινιών αλλα και με βάση ένα ξεχωριστό αρχείο το οποίο περιέχει τις αξιολογίσεις του χρήστη για ενα πλήθος διαφορετικών ταινιών.Συγκεκριμένα για το πλήθος των διαφορετικών ταινίων που έχει βαθμολογίσει βρίσκουμε όλες τις ταινίες που είναι τουλάχιστον similar 1 με αυτή. Καθε ταινία που προκειται να προταθεί έχει ενα recomendation score που ορίζεται ως το βάρος που έχει η ταινία σχετικά με την ομοιότητα της με αυτή που έχει βαθμολογίσει ο χρήστης επι το βάρος βαθμολογίας που έχει δώσει ο χρήστης σε αυτή τη ταινια και αθροίζουμε για όλες τις βαθμολογημένες ταινίες.

Τα βάρη ομοιότητας έχουν οριστει αναλογικά και συγκεκριμένα:

similarity type 5 weight = 16 

similarity type 4 weight = 12

similarity type 3 weight = 8 

similarity type 2 weight = 4 

similarity type 1 weight = 1

Τα βάρη σχετικά με τη βαθμολόγιση της ταινίας έχουν οριστεί με διαφορετικη λογική με την έννοια οτι θέλουμε οι ταινίες με βαθμό πεντε να παίζουν μεγάλο ρόλο , οι ταινίες με βαθμό 3 να είναι ουδέτερες ενώ οι ταινίες με βαθμό 1 να έχουν αρνητικό αντίκτυπο.

Συγκεκριμένα έχουμε:

voting weight 5 = 10 

voting weight 4 = 5 

voting weight 3 = 0 

voting weight 2 = - 5

voting weight 1 = - 10 






## Προτάσεις για τον 1ο user


In [ ]:
user = "/content/drive/My Drive/AI_ex2/user1.csv"
ratings = pd.read_csv(user) 

# Preview the first 5 lines of the loaded data 
ratings.head()

,movie_title,rating
0,Heavy Metal,4
1,The Croods,5
2,Watchmen,4
3,The Matrix,4
4,The Emperors New Groove,5


In [ ]:
def make_movie_score(ratings,mov):
  similarity_weights = {"similarity_5": 16, "similarity_4":12, "similarity_3": 8,"similarity_2":4 ,"similarity_1": 1}
  rating_weight = {5: 10, 4: 5, 3: 0, 2: -5, 1: -10}

  sc = {m: 0 for m in  mov}
  for row in ratings.itertuples(index=True, name='Pandas'):
      movie = clean_text(getattr(row, 'movie_title'))
      rating = getattr(row, 'rating')    
      
      q = prolog.query("find_similar_movies_5('" + movie + "',M)")
      for soln in q:
          sc[soln["M"]] += similarity_weights['similarity_5'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
      q.close()
      
      q = prolog.query("find_similar_movies_4('" + movie + "',M)")
      for soln in q:
          sc[soln["M"]] += similarity_weights['similarity_4'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
      q.close()
      
      q = prolog.query("find_similar_movies_3('" + movie + "',M)")
      for soln in q:
          sc[soln["M"]] += similarity_weights['similarity_3'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
      q.close()
      
      q = prolog.query("find_similar_movies_2('" + movie + "',M)")
      for soln in q:
          sc[soln["M"]] += similarity_weights['similarity_2'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
      q.close()

      q = prolog.query("find_similar_movies_1('" + movie + "',M)")
      for soln in q:
          sc[soln["M"]] += similarity_weights['similarity_1'] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
      q.close()
  return sc

In [ ]:
def recommender(movie_score):
    movies_score = {k: v for k, v in sorted(movie_score.items(), key=lambda item: item[1], reverse = True)}
    return [m for m in movies_score][:10]



In [ ]:
mov_scores = make_movie_score(ratings, movie_names)



In [ ]:
rec = recommender(mov_scores)

print("##########################################")
print("movies recomended for 1st user")
print("###########################################")
for i in rec:
  print("movie:  ",i,"score: ",mov_scores[i])

##########################################
movies recomended for 1st user
###########################################
movie:   The Unborn score:  1780
movie:   Home score:  1725
movie:   Teenage Mutant Ninja Turtles score:  1540
movie:   Casino Royale score:  1360
movie:   Space Chimps score:  1345
movie:   Tangled score:  1125
movie:   Chicken Little score:  1105
movie:   Star Wars: Episode I - The Phantom Menace score:  1025
movie:   The Ant Bully score:  1025
movie:   Alpha and Omega 4: The Legend of the Saw Toothed Cave score:  1020


## Προτάσεις για τον δεύτερο user

In [ ]:
user2 = "/content/drive/My Drive/AI_ex2/user2.csv"
ratings2 = pd.read_csv(user2) 

# Preview the first 5 lines of the loaded data 
ratings2.head()

,movie_title,rating
0,Loser,1
1,All That Jazz,2
2,The Sting,2
3,Torque,3
4,Project X,1


In [ ]:
mov_scores2 = make_movie_score(ratings2, movie_names)

In [ ]:
rec2 = recommender(mov_scores2)

print("##########################################")
print("movies recomended for 2nd user")
print("###########################################")
for i in rec2:
  print("movie:  ",i,"score: ",mov_scores2[i])

##########################################
movies recomended for 2nd user
###########################################
movie:   Jack Reacher score:  5800
movie:   Lucky Number Slevin score:  4080
movie:   Trance score:  3840
movie:   Snakes on a Plane score:  3560
movie:   My Soul to Take score:  3480
movie:   The Gunman score:  3240
movie:   Mercury Rising score:  3120
movie:   Disturbia score:  2960
movie:   Sabotage score:  2960
movie:   Blood Work score:  2940


## ΚΑΤΑΣΚΕΥΗ ΔΙΚΩΝ ΜΑΣ USERS


In [ ]:
import csv
import numpy as np

def create_my_user(liked, notLiked, filename, num_of_liked =20, num_of_notliked = 20):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["movie_title","rating"])
        likes = np.random.choice(liked, num_of_liked)
        not_likes = np.random.choice(notliked, num_of_notliked)
        for i in  likes:
            grade = np.random.choice([5,4,3], p = [0.5, 0.4, 0.1])
            writer.writerow([i, grade])
        for i in not_likes:
            grade = np.random.choice([1, 2], p = [0.6, 0.4])
            writer.writerow([i, grade])

Χρησιμοποιούμε έναν user που του αρέσουν ταινίες fantasy που έχει σκηνοθετήσει ο James Cameron και δεν του αρέσουν τα θρίλερ και έναν δεύτερο user που του αρέσουν ταινίες δράσης και horror και δεν του αρέσουν οι κωμωδίες.

In [ ]:
# prolog.consult("db.pl")
#Κατασκευάστε τα κατηγορήματα για τις ταινίες που σας αρέσουν και για αυτές που δεν σας αρέσουν
#Αν μια ταινία ανήκει σε μια ευρίτερη κατηγορία κατεσκευάστε την πρώτα
prolog.assertz("(fantasy(X) :- genre(X, 'Sci-Fi'); genre(X, 'Fantasy'))")
prolog.assertz("(thriller(X) :- genre(X, 'Horror'); genre(X, 'Thriller'))")
prolog.assertz("(action(X)    :- genre(X,'Action'),genre(X,Adventure))")
prolog.assertz("(funny(X)     :- genre(X,'Comedy'))")
prolog.assertz("(dark(X)      :- genre(X,'Horror'))")

prolog.assertz("(liked(X) :- fantasy(X), director(X, 'James Cameron'))")
prolog.assertz("(notLiked(X) :- thriller(X))")

In [ ]:
prolog.assertz("(liked1(X)    :- dark(X),action(X))")
prolog.assertz("(notliked1(X) :- funny(X))")

In [ ]:
q = prolog.query("liked(M)")
ans = []
for soln in q:
    ans.append(soln["M"])
liked = list(set(ans))

q = prolog.query("notLiked(M)")
ans = []
for soln in q:
    ans.append(soln["M"])
notliked = list(set(ans))

q = prolog.query("liked1(M)")
ans = []
for soln in q:
    ans.append(soln["M"])
liked1 = list(set(ans))

q = prolog.query("notliked1(M)")
ans = []
for soln in q:
    ans.append(soln["M"])
notliked1 = list(set(ans))

create_my_user(liked, notliked, filename= "/content/drive/My Drive/AI_ex2/myuser.csv", num_of_liked = 20, num_of_notliked = 20)
create_my_user(liked1, notliked1, filename= "/content/drive/My Drive/AI_ex2/myuser1.csv", num_of_liked = 20, num_of_notliked = 20)

In [ ]:
my_user = "/content/drive/My Drive/AI_ex2/myuser.csv"
my_ratings = pd.read_csv(my_user) 
my_scores = make_movie_score(my_ratings, movie_names)
my_recs = recommender(my_scores)
print(my_scores)

for i in my_recs:
  print("movie:  ",i, "Score :",my_scores[i],"\n")

{'Avatar': 4405, 'Pirates of the Caribbean: At Worlds End': -110, 'Spectre': -150, 'The Dark Knight Rises': -110, 'Star Wars: Episode VII - The Force Awakens': 0, 'John Carter': 430, 'Spider-Man 3': -200, 'Tangled': -50, 'Avengers: Age of Ultron': -50, 'Harry Potter and the Half-Blood Prince': -50, 'Batman v Superman: Dawn of Justice': -50, 'Superman Returns': -90, 'Quantum of Solace': -130, 'Pirates of the Caribbean: Dead Mans Chest': 30, 'The Lone Ranger': -50, 'Man of Steel': -50, 'The Chronicles of Narnia: Prince Caspian': -90, 'The Avengers': -200, 'Pirates of the Caribbean: On Stranger Tides': -190, 'Men in Black 3': 410, 'The Hobbit: The Battle of the Five Armies': -110, 'The Amazing Spider-Man': -110, 'Robin Hood': -50, 'The Hobbit: The Desolation of Smaug': -110, 'The Golden Compass': -50, 'King Kong': -570, 'Titanic': 1570, 'Captain America: Civil War': -50, 'Battleship': -210, 'Jurassic World': -170, 'Skyfall': -320, 'Spider-Man 2': -290, 'Iron Man 3': -50, 'Alice in Wonderl

In [ ]:
my_user1 = "/content/drive/My Drive/AI_ex2/myuser1.csv"
my_ratings1 = pd.read_csv(my_user1) 
my_scores1 = make_movie_score(my_ratings1, movie_names)
my_recs1 = recommender(my_scores1)


for i in my_recs1:
  print("movie:  ",i, "Score :",my_scores1[i],"\n")

movie:   Snowpiercer Score : 1650 

movie:   Hellboy II: The Golden Army Score : 1330 

movie:   The Host Score : 1040 

movie:   Blade II Score : 1010 

movie:   In Time Score : 950 

movie:   The Watch Score : 920 

movie:   Dylan Dog: Dead of Night Score : 890 

movie:   The Scorpion King Score : 870 

movie:   Ultramarines: A Warhammer 40,000 Movie Score : 810 

movie:   A Nightmare on Elm Street 4: The Dream Master Score : 770 

